In [3]:
!pip install deims
!pip install xmltodict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import uuid
import json
import xmltodict
import deims
import datetime
import os

In [19]:
with open('getcapabilities_1.3.0.xml') as fd:
    doc = xmltodict.parse(fd.read())

for layer in doc['WMS_Capabilities']['Capability']['Layer']['Layer']:

    # date stamp of metadata update date
    now = datetime.datetime.now()

    # fetch EPSG code
    epsg_code = ''
    for value in layer['CRS']:
        if "EPSG:" in value:
            epsg_code = value[5:]

    # boundaries
    westBoundLongitude = float(layer['EX_GeographicBoundingBox']["westBoundLongitude"])
    eastBoundLongitude = float(layer['EX_GeographicBoundingBox']["eastBoundLongitude"])
    southBoundLatitude = float(layer['EX_GeographicBoundingBox']["southBoundLatitude"])
    northBoundLatitude = float(layer['EX_GeographicBoundingBox']["northBoundLatitude"])

    longitude_centre = (westBoundLongitude+eastBoundLongitude)/2.0
    latitude_centre = (northBoundLatitude+southBoundLatitude)/2.0

    keywords_to_be_printed = ["eLTER"]
    record_uuid = None
    authors = None
    so_reference = None
    # process keywords for site information and temporal extent
    for keyword in layer['KeywordList']['Keyword']:
        uri = None

        if "id: " in keyword:
            record_uuid = keyword[4:]
            continue

        if "project: " in keyword:
            if "e-shape" in keyword:
                project = {
                    "name": "e-shape",
                    "PID": "https://doi.org/10.3030/820852"
                }
                continue

            if "agame" in keyword:
                project = {
                    "name": "AGAME",
                    "PID": "https://www.umweltbundesamt.at/en/services/agame"
                }
                continue

        if "variable: " in keyword:
            data_product_type = keyword[10:]
            if data_product_type == "Phenology":
                keyword = "Phenology"
                uri = "https://vocabs.lter-europe.net/EnvThes/21647"
                authors = [{
                  "fullName": "Saverio Vicario",
                  "givenName": "Saverio",
                  "familyName": "Vicario",
                  "email": "saverio.vicario@iia.cnr.it",
                  "type": "ContactPerson",
                  "ids": [{
                      "id": "https://orcid.org/0000-0003-1140-0483",
                      "schema": "Orcid",
                    }]
                }]

            if data_product_type == "Hydroperiod":
                keyword = "Hydroperiod"
                authors = [{
                  "fullName": "Ioannis Manakos",
                  "givenName": "Ioannis",
                  "familyName": "Manakos",
                  "email": "imanakos@iti.gr",
                  "type": "ContactPerson",
                  "ids": [{
                      "id": "https://orcid.org/0000-0001-6833-294X",
                      "schema": "Orcid",
                    }]
                }]

            if data_product_type == "Gross Primary Production":
                keyword = "Gross Primary Production"
                uri = "http://vocabs.lter-europe.net/EnvThes/21074"
                so_reference = {
                    "name": "Gross Primary Production",
                    "url": "https://vocabs.lter-europe.net/so/090"
                },
                authors = [{
                    "fullName": "Mario Alberto Fruentes-Monjaraz",
                    "givenName": "Alberto",
                    "familyName": "Fuentes-Monjaraz",
                    "type": "ContactPerson",
                    "email": "mario.fuentesmonjaraz@deltares.nl",
                    "ids": [{
                        "id": "https://orcid.org/0000-0002-0711-8867",
                        "schema": "Orcid",
                    }]
                },
                {
                    "fullName": "Anna Spinosa",
                    "givenName": "Anna",
                    "familyName": "Spinosa",
                    "type": "ContactPerson",
                    "email": "anna.spinosa@deltares.nl",
                    "ids": [{
                        "id": "https://orcid.org/0000-0002-7921-5922",
                        "schema": "Orcid",
                    }]
                },
                {
                  "fullName": "Valeria Mobilia",
                  "givenName": "Valeria",
                  "familyName": "Mobilia",
                  "type": "ContactPerson",
                  "email": "valeria.mobilia@deltares.nl",
                  "ids": [{
                      "id": "https://orcid.org/0000-0001-9370-598X",
                      "schema": "Orcid",
                    }]
                }]

            if data_product_type == "snowcover":
                uri = "https://vocabs.lter-europe.net/EnvThes/21559"
                keyword = "snowCover"
                authors = [{
                    "fullName": "Maria Adamo",
                    "givenName": "Maria",
                    "familyName": "Adamo",
                    "type": "ContactPerson",
                    "email": "adamo@iia.cnr.it",
                    "ids": [{
                        "id": "https://orcid.org/0000-0003-3030-4884",
                        "schema": "Orcid",
                    }]
                },
                {
                    "fullName": "Chiara Richiardi",
                    "givenName": "Chiara",
                    "familyName": "Richiardi",
                    "type": "ContactPerson",
                    "email": "chiara.richiardi@iia.cnr.it",
                    "ids": [{
                        "id": "https://orcid.org/0000-0002-2370-7768",
                        "schema": "Orcid",
                    }]
                }]

        if "site: " in keyword:
            deims_suffix = keyword[6:]
            #print(site_record)
            uri = "https://deims.org/" + deims_suffix
            site_record = deims.getSiteById(deims_suffix)

            try:
                site_name = site_record["title"]
            except:
                print(layer['Title'])
                print(deims_suffix)
            continue

        if "https://doi.org/" in keyword:
            doi = {
                "name": "B2Share",
                "description": "Download data on B2Share",
                "url": keyword
            }

        if "time: " in keyword:
            time_string = keyword[6:]

            if "-" in time_string:
                time_values = time_string.split('-')
                period = [{
                    "startDate": time_values[0] + '-01-01',
                    "endDate": time_values[1] + '-12-31'
                }]

            else:
                period = [{
                    "startDate": time_string + '-01-01',
                    "endDate": time_string + '-12-31'
                }]
            continue

        if uri:
            formatted_keyword = {
                "name": keyword,
                "url": uri
            }
        else:
            formatted_keyword = {
                "name": keyword,
                "url": ""
            }

        keywords_to_be_printed.append(formatted_keyword)

    #print(deims_suffix)

    record = {
        "metadata": {
            "datasetIds": [{
                "sourceName": "eLTER spatial.io",
                "identifier": record_uuid,
                "type": "PID"
            }],
            "titles": [{
                "language": "eng",
                "text": layer['Title']
            }],
            "descriptions": [{
                "type": "Abstract",
                "language": "eng",
                "description": layer['Abstract'] # we could add a sentence that the metadata record was generated automatically
            }],
            "keywords": keywords_to_be_printed,
            "publicationDate": datetime.datetime.now().strftime('%Y-%m-%d'),
            "language": "english",
            "SOReference": so_reference,
            "authors": authors,
            "geoLocations": [
                {
                    "EX_GeographicDescription": site_record["title"],
                    "Point": {
                        "longitude": longitude_centre,
                        "latitude": latitude_centre
                    }
                },
                {
                      "EX_GeographicDescription": site_record["title"],
                      "EX_GeographicBoundingBox": {
                           "westBoundLongitude": westBoundLongitude,
                           "eastBoundLongitude": eastBoundLongitude,
                           "southBoundLatitude": southBoundLatitude,
                           "northBoundLatitude": northBoundLatitude 
                      }
                 }
            ],
            "temporalCoverages": period,
            "geoServerInfo": {
                "serviceType": "WMS",
                "mapData": [
                    {
                        "path": "https://spatialnode.elter.cerit-sc.cz/geoserver/ows?version=1.3.0",
                        "epsgCode": epsg_code,
                        "type": "POLYGON",
                        "features": {
                            "name": layer['Name'],
                            "label": layer['Title'],
                            "style": {
                                "colour": "#000000"
                            }
                        },
                        "bytetype": False
                    }
                ]
            },
            "project": project,
            "siteReference": [{
                "name": site_record["title"],
                "PID": deims_suffix
            }],
        },
        "files": {
            "enabled": False
        }
    }
    
    filename = record_uuid + ".json"
    base_dir = os.getcwd() + '/files_for_catalogue'
    destination_path = os.path.join(base_dir, filename)

    with open(destination_path, "w+") as f:
        json.dump(record, f)


In [20]:
# to zip all generated files
import shutil
shutil.make_archive("files_for_catalogue", 'zip', base_dir)

'/home/wohner/work/files_for_catalogue.zip'

In [27]:
#!pip install dotenv
#import os
#import json
#import aiohttp
#import asyncio
#import logging

#from dotenv import load_dotenv

load_dotenv()

log_file = 'app.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file)
    ]
)
logger = logging.getLogger(__name__)

TOKEN = os.getenv('TOKEN')
API_BASE = os.getenv('API_BASE')
MAX_CONCURRENT_REQUESTS = 4  # Adjust


async def send_publish_request(session, url, retries=3, backoff=1):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {TOKEN}'
    }
    async with session.post(url, headers=headers) as response:
        response_text = await response.text()
        if response.status == 202:
            logger.info(f"Success published - {url}")
        elif response.status in {502, 503} and retries > 0:
            await asyncio.sleep(backoff)
            await send_publish_request(session, url, retries - 1, backoff * 2)
        else:
            logger.error(f"Publish request status: {response.status}, Response: {response_text}")


async def send_create_request(session, url, json_data, filename, retries=3, backoff=1):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {TOKEN}'
    }

    try:
        async with session.post(url, json=json_data, headers=headers) as response:
            response_text = await response.text()
            if response.status == 201:
                logger.info(f"Success from {filename}: {response.status}, Response: {response_text}")
                try:
                    response_json = await response.json()
                    publish_link = response_json.get('links', {}).get('publish', None)
                    if publish_link:
                        await send_publish_request(session, publish_link, retries, backoff)

                except json.JSONDecodeError:
                    logger.error(f"Error decoding JSON response from {filename}")
            elif response.status in {502, 503} and retries > 0:
                logger.warning(f"Server error ({response.status}) from {filename}. Retrying in {backoff} seconds...")
                await asyncio.sleep(backoff)
                await send_create_request(session, url, json_data, filename, retries - 1, backoff * 2)
            else:
                logger.error(f"Failed from {filename}: {response.status}, Response: {response_text}")
    except aiohttp.ClientError as e:
        logger.error(f"Request failed for {filename}: {e}")


async def load_and_send_json_files():
    directory = './json_resources'

    if not API_BASE:
        logger.error("API_BASE environment variable is not set in the .env file.")
        return

    if not TOKEN:
        logger.error("TOKEN environment variable is not set in the .env file.")
        return

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    async def bound_send_post_request(session, url, json_data, filename, retries=3, backoff=1):
        async with semaphore:
            await send_create_request(session, url, json_data, filename, retries, backoff)

    async with aiohttp.ClientSession() as session:
        tasks = []
        for filename in os.listdir(directory):
            if filename.endswith('.json'):
                file_path = os.path.join(directory, filename)
                try:
                    with open(file_path, 'r') as json_file:
                        json_data = json.load(json_file)
                        logger.info(f"Preparing to send data from {filename}...")
                        task = bound_send_post_request(session, API_BASE, json_data, filename)
                        tasks.append(task)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding JSON from {filename}: {e}")
                except Exception as e:
                    logger.error(f"An error occurred with {filename}: {e}")

        await asyncio.gather(*tasks)


if __name__ == "__main__":
    asyncio.run(load_and_send_json_files())


Exception ignored in: <coroutine object load_and_send_json_files at 0x7f777d0f42e0>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object load_and_send_json_files at 0x7f777d0f42e0>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'


NameError: name 'load_dotenv' is not defined